In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
### EDA

def df_check(df):
    ret_df = pd.DataFrame()
    feature_names=['missing count', 'u_count', 'min','max','mean', 'median', 'std']
    i = 0
    for s in df:
        s_list = []
        s_list.append(s)
        #s_list.append((df[s].isna().sum()*100)/len(df[s]))
        s_list.append(df[s].isna().sum())
        s_list.append(len(df[s].unique()))
        try:
            df[s].astype('float64')
            s_list.append(df[s].min())
            s_list.append(df[s].max())
            s_list.append(df[s].mean())
            s_list.append(df[s].median())
            s_list.append(df[s].std())
        except:
            for _ in range(5):
                s_list.append(np.NaN)
        finally:
            ret_df.insert(i, s, pd.Series(s_list[1:]))
            i = i+1
    x = ret_df.T
    x.columns = feature_names
    return x

### read raw data
pd_train = pd.read_csv('train.csv')
pd_test = pd.read_csv('test.csv')

### check shape, features, basic statistics
r = df_check(pd_train)
#print(r)
#print(df_check(pd_test))

### EDA

#### Train

* 학습 타겟용 'Survived' -  feature / dimension / df column 있음.
* 'Age' (177/891), 'Cabin' (687/891), 'Embarked' (2/891) 결측치.


### Test
* 'Age' (86/418), 'Cabin' (327/418), 'Fare (1/418)' 결측치.

### 기타
* 'Cabin', 'Name', 'Ticket' 사용않기로 함.


### todo
* outlier check

In [ ]:
### missing value -- mean, median, 고빈도 값으로 대체

### read raw data
pd_train = pd.read_csv('train.csv')
pd_test = pd.read_csv('test.csv')

### align train, test dataframes
y_train = pd_train['Survived'].values
pd_train.drop('Survived', axis=1, inplace=True)

g1 = pd_train.groupby(['Pclass', 'Sex'])
g2 = pd_test.groupby(['Pclass', 'Sex'])

### missing value: Age
# mean, median(50%) 가까움, mean 값으로 대체
#print(g1['Age'].describe())

df_age_mean = pd.DataFrame()
m1 = g1['Age'].mean()
m2 = g2['Age'].mean()

df_age_mean.insert(0, 'pd_train', m1)
df_age_mean.insert(1, 'pd_test', m2)

pd_data = pd.concat([pd_train, pd_test], ignore_index=True)
g3 = pd_data.groupby(['Pclass', 'Sex'])
m3 = g3['Age'].mean()
df_age_mean.insert(2, 'pd_data', m3)

#print(df_age_mean)

f1 = pd_data['Age'].isna()
f2 = pd_data['Age'].notna()
#print(pd_data[f1])

### missing value: Fare 
# mean, median(50%) 차이. outlier영향 있을것으로 예상. median 값으로 대체
#print(g1['Fare'].describe())

df_fare_desc = g1['Fare'].describe()

### missing value: Embarked
# 빈도수 높은 top 값으로 대체
#print(g1['Embarked'].describe())

df_embarked_desc = g1['Embarked'].describe()

# drop unused features
pd_data = pd_data.drop(['Name', 'Cabin', 'Ticket'], axis=1)

def df_fill_missing(df):
    for i in range(len(df)):
        row = df.iloc[i, :]
        row_pclass = row['Pclass']
        row_sex = row['Sex']
        row_fare = row['Fare']
        
        if np.isnan(row['Age']):
            df.loc[i, 'Age'] = df_age_mean.loc[(row_pclass, row_sex), 'pd_data']
            
        if np.isnan(row['Fare']):
            df.loc[i, 'Fare'] = df_fare_desc.loc[(row_pclass, row_sex), '50%']
        
        if row['Embarked'] is np.NaN:
            df.loc[i, 'Embarked'] = df_embarked_desc.loc[(row_pclass, row_sex), 'top']

def df_catvalue(df):
    for s in df:
        try:
            df[s].astype('float64')
        except:
            if len(df[s].unique())<10:
                df[s] = df[s].astype('category').cat.codes
            else:
                pass
            
df_fill_missing(pd_data)
df_catvalue(pd_data)
df_check(pd_data)
#print(pd_data)

In [ ]:
df_age_predict = pd.DataFrame()

### missing value, age -- linear regression
df_age_predict.insert(0, 'mean', pd_data[f1]['Age'])
#print(pd_data[f2])
#print(df_age_predict)

age_x_train = pd_data[f2].drop('Age', axis=1)
age_y_train = pd_data[f2].loc[:, 'Age']
age_x_predict = pd_data[f1].drop('Age', axis=1)

#print(age_x_predict)

### scale
def minmax(s):
    return (s-s.min())/(s.max()-s.min())

for dset in [age_x_train, age_x_predict]:
    for c in dset.iloc[:, 1:]:
        dset[c] = minmax(dset[c])
        
#print(age_y_train)

lr = LinearRegression().fit(age_x_train, age_y_train)
p = lr.predict(age_x_predict)

df_age_predict.insert(1, 'lr_predict', pd.Series(p, index = df_age_predict.index))
#print(df_age_predict)

for i in df_age_predict.index:
    pd_data.loc[i, 'Age'] = df_age_predict.loc[i, 'lr_predict']

In [ ]:
### scale - minmax, exclude PassengerId

def minmax(s):
    return (s-s.min())/(s.max()-s.min())

for c in pd_data.iloc[:, 1:]:
    pd_data[c] = minmax(pd_data[c])
#print(pd_data)

In [ ]:
### split
train_len = len(y_train)

X_train = pd_data.iloc[:train_len, 1:]
X_test = pd_data.iloc[train_len:, 1:]

print(X_train.shape, y_train.shape, X_test.shape)
#print(X_train.describe())

In [ ]:
### DNN

model = keras.Sequential([
    keras.layers.Dense(7, input_shape=(X_train.shape[1],), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = keras.losses.BinaryCrossentropy(),
             optimizer = keras.optimizers.Adam(),
             metrics=['accuracy'])

hist = model.fit(X_train, y_train, epochs = 50, verbose=0)
model.evaluate(X_train, y_train)

In [ ]:
lr = LogisticRegression().fit(X_train, y_train)
lr.score(X_train, y_train)

In [ ]:
dt = DecisionTreeClassifier().fit(X_train, y_train)
dt.score(X_train, y_train)

In [ ]:
rf = RandomForestClassifier().fit(X_train, y_train)
rf.score(X_train, y_train)

p = rf.predict(X_test)